In [1]:
import pandas as pd
import re

In [2]:
import sys
sys.path.append('../')

In [3]:
from metaexplainercode import metaexplainer_utils
from metaexplainercode import codeconstants

In [4]:
user_responses = codeconstants.SYNTHESIS_FOLDER + '/eval_files/user_responses/'

In [5]:
all_subfolders = metaexplainer_utils.get_subfolders_in_folder(user_responses)

In [6]:
system_responses_list = []
per_explanation_response_list = []
user_responses_list = []

per_evaluation_pattern = 'evaluation_set_\d+.xlsx'

for response_folder in all_subfolders:
    system_responses_list.append(pd.read_excel(response_folder + '/OverallSystemEvaluationSet.xlsx'))
    user_responses_list.append(pd.read_excel(response_folder + '/UserFamiliarityQuestionnaire.xlsx'))

    all_files = metaexplainer_utils.get_files_in_folder(response_folder)

    for eval_file in all_files:
        eval_file = eval_file.split('/')[-1]
        matched_eval = re.match(per_evaluation_pattern, eval_file)

        if matched_eval is not None:
            matched_eval_file = matched_eval[0]
            #print('Match', matched_eval_file)
            per_explanation_response_list.append(pd.read_excel(response_folder + '/' + matched_eval_file, index_col=[0], engine="openpyxl"))
            break

In [43]:
per_explanation_response_df = pd.concat(per_explanation_response_list, ignore_index=True)


In [62]:
#dropping duplicate column

per_explanation_response_df.rename({ele:'The explanation is sufficiently complete.' for ele in list(per_explanation_response_df.columns) if 'The explanation is sufficiently complete.' in ele},axis=1,inplace=True)
per_explanation_response_df = per_explanation_response_df.groupby(per_explanation_response_df.columns, axis=1).agg(np.max)

In [63]:
per_explanation_response_df.columns

Index(['Explanation of Explainer Outputs', 'Explanation of Matched Subset',
       'Feature groups in Question',
       'I understand this AI system correctly due to the explanation.',
       'Metrics', 'Predicted Explanation Type', 'Question',
       'The explanation is sufficiently complete.',
       'This explanation has sufficient detail.',
       'This explanation is useful to my goals.'],
      dtype='object')

In [64]:
per_explanation_response_df.groupby('Predicted Explanation Type').mean()

,I understand this AI system correctly due to the explanation.,The explanation is sufficiently complete.,This explanation has sufficient detail.,This explanation is useful to my goals.
Predicted Explanation Type,,,,
Case Based Explanation,0.571429,0.214286,0.642857,0.214286
Contrastive Explanation,0.815789,0.631579,0.868421,0.710526
Counterfactual Explanation,0.642857,0.500000,0.809524,0.714286
Data Explanation,0.980000,0.840000,1.060000,1.000000
Rationale Explanation,0.682540,0.412698,0.523810,0.619048


Index(['Explanation of Explainer Outputs', 'Explanation of Matched Subset',
       'Feature groups in Question',
       'I understand this AI system correctly due to the explanation.',
       'Metrics', 'Predicted Explanation Type', 'Question',
       'The explanation is sufficiently complete.',
       'This explanation has sufficient detail.',
       'This explanation is useful to my goals.'],
      dtype='object')

In [8]:
#which dimensions are from trust, curiosity and satisfication

system_responses_df = pd.concat(system_responses_list)

system_responses_df.groupby(['Evaluation Question']).mean()

,Likert Rating
Evaluation Question,
I am confident in the MetaExplainer. I feel that it works well. [Error frequency],2.285714
I like using the system for decision making. [Satisfication],2.214286
I want to know more about what the AI did.,4.214286
I want to know what the AI would have done if something had been different.,4.214286
I want to know why the AI did not make some other decision.,4.071429
I want to understand what the AI will do next.,3.928571
I was surprised by the AI’s actions and want to know what I missed.,3.071429
The MetaExplainer can perform the task better than a novice human user.,2.714286
The outputs of the MetaExplainer are predictable. [Memorability],3.428571


In [9]:
import numpy as np

user_details_df = pd.concat(user_responses_list)
user_details_df.head(20)

,Question,Answer,Unnamed: 2
0,How familiar are you with Explainable AI? (Lik...,5,NaN
1,What expalinable AI tools / methods have you u...,"LIME, SHAP, TDA, Feature Attribution",NaN
2,How familiar are you with Diabetes? (Likert ra...,2,NaN
3,Were the questions too specific to Diabetes or...,1,NaN
0,How familiar are you with Explainable AI? (Lik...,4,NaN
1,What expalinable AI tools / methods have you u...,Counterfactual Explainability,NaN
2,How familiar are you with Diabetes? (Likert ra...,4,NaN
3,Were the questions too specific to Diabetes or...,2,NaN
0,How familiar are you with Explainable AI? (Lik...,3,NaN
1,What expalinable AI tools / methods have you u...,Never used any myself,NaN


In [10]:
user_details_df['Question'].unique()

array(['How familiar are you with Explainable AI? (Likert Rating)',
       'What expalinable AI tools / methods have you used before?',
       'How familiar are you with Diabetes? (Likert rating)',
       'Were the questions too specific to Diabetes or general enough? (0 - No, 1 - Yes, 2 - Maybe)'],
      dtype=object)